In [1]:
import numpy as np
import re
from itertools import permutations
import time

In [2]:
with open("input_21.txt", "r") as fh:
    sequences = [s.strip() for s in fh.readlines()]
sequences

['129A', '974A', '805A', '671A', '386A']

In [3]:
num_coordinates = {
    'A': (3,2),
    '0': (3,1),
    '1': (2,0),
    '2': (2,1),
    '3': (2,2),
    '4': (1,0),
    '5': (1,1),
    '6': (1,2),
    '7': (0,0),
    '8': (0,1),
    '9': (0,2),
}

arrow_coordinates = {
    'A': (0, 2),
    '^': (0, 1),
    'v': (1, 1),
    '<': (1, 0),
    '>': (1, 2)
}


def step_on_numpad(from_btn, to_btn):
    '''Find single (not only) optimal step through numpad'''
    from_coord = num_coordinates[from_btn]
    to_coord = num_coordinates[to_btn]
    r_steps, c_steps = to_coord[0]-from_coord[0], to_coord[1]-from_coord[1]
    sequence = []

    if r_steps < 0:
        # First up
        for i in range(-r_steps):
            sequence.append("^")
        # Then left or right
        for i in range(abs(c_steps)):
            sequence.append("<" if c_steps < 0 else ">")
    else:
        # First left or right
        for i in range(abs(c_steps)):
            sequence.append("<" if c_steps < 0 else ">")
        # Then down
        for i in range(r_steps):
            sequence.append("v")
    sequence.append("A")
    return sequence


def step_on_arrowpad(from_btn, to_btn):
    '''Find single (not only) optimal step through arrow pad'''
    from_coord = arrow_coordinates[from_btn]
    to_coord = arrow_coordinates[to_btn]
    r_steps, c_steps = to_coord[0]-from_coord[0], to_coord[1]-from_coord[1]
    sequence = []

    if r_steps >= 0:
        # First down
        for i in range(abs(r_steps)):
            sequence.append("v")
        # Then left or right
        for i in range(abs(c_steps)):
            sequence.append("<" if c_steps < 0 else ">")
    else:
        # First left or right
        for i in range(abs(c_steps)):
            sequence.append("<" if c_steps < 0 else ">")
        # Then up
        for i in range(abs(r_steps)):
            sequence.append("^")
    sequence.append("A")
    return sequence


def convert_sequence(seq, numeric_pad=True):
    '''Find single (not only) optimal solution of sequence on one of the pads.'''
    seq_out = []
    seq = ["A"] + list(seq)
    for i in range(len(seq)-1):
        seq_out += step_on_numpad(seq[i], seq[i+1]) if numeric_pad else step_on_arrowpad(seq[i], seq[i+1])

    return "".join(seq_out)


def get_legal_solutions(seq, numeric_pad=True):
    '''Get all legal optimal solutions of a sequence.'''
    # Calculate first solution
    seq = convert_sequence(seq, numeric_pad)
    
    # Get all possible solutions
    perms = [list(permutations(s)) for s in seq.split("A")]
    all_solutions = set()
    for p in perms:
        temp_solutions = set()
        for pi in p:
            if not len(all_solutions):
                temp_solutions.add("".join(pi) + "A")
            else:
                for s in all_solutions:
                    temp_solutions.add(s + "".join(pi) + "A")
        all_solutions = temp_solutions.copy()

    # Check if solution is legal
    if numeric_pad:
        start_loc = (3, 2)
        forbidden_loc = (3, 0)
    else:
        start_loc = (0, 2)
        forbidden_loc = (0, 0)
    legal_solutions = []
    for solution in all_solutions:
        legal = True
        r_loc, c_loc = start_loc
        for c in solution:
            match(c):
                case "v": r_loc += 1
                case "^": r_loc -= 1
                case "<": c_loc -= 1
                case ">": c_loc += 1
            if (r_loc, c_loc) == forbidden_loc:
                legal = False
                break
        if legal:
            legal_solutions.append(solution[:-1]) # trailing A should be removed
    
    return legal_solutions

In [4]:
# 21a
sum_complexities = 0
for sequence in sequences:
    # Calculate initial sequences
    seqs = get_legal_solutions(sequence, True)
    # Calculate second sequences
    new_seqs = []
    for s in seqs:
        new_seqs += get_legal_solutions(s, False)
    seqs = new_seqs.copy()
    # Calculate third sequences
    lengths = [len(convert_sequence(s, False)) for s in seqs]
    
    # Calculate complexity
    numeric_part = int(re.findall(r"(\d+)", sequence)[0])
    sum_complexities += numeric_part * min(lengths)
print("21a:", sum_complexities)

21a: 213536


In [5]:
# 21b - functions
# Format: k=(from, to), v=possible paths
arrow_paths = {
    ("A", "A"): ["A"],
    ("A", "<"): ["v<<A", "<v<A"],
    ("A", ">"): ["vA"],
    ("A", "^"): ["<A"],
    ("A", "v"): ["v<A", "<vA"],
    ("<", "A"): [">>^A", ">^>A"],
    ("<", "<"): ["A"],
    ("<", ">"): [">>A"], # longer route possible
    ("<", "^"): [">^A"], # longer route possible
    ("<", "v"): [">A"],
    (">", "A"): ["^A"],
    (">", "<"): ["<<A"],
    (">", ">"): ["A"],
    (">", "^"): ["^<A", "<^A"],
    (">", "v"): ["<A"],
    ("^", "A"): [">A"],
    ("^", "<"): ["v<A"],
    ("^", ">"): [">vA", "v>A"],
    ("^", "^"): ["A"],
    ("^", "v"): ["vA"],
    ("v", "A"): [">^A", "^>A"],
    ("v", "<"): ["<A"],
    ("v", ">"): [">A"],
    ("v", "^"): ["^A"],
    ("v", "v"): ["A"],
}

# Compose all possible best arrow paths, since one will be most optimal
arrow_path_combinations = [{}]
for k in arrow_paths:
    if len(arrow_paths[k]) == 1:
        for apc in arrow_path_combinations:
            apc[k] = arrow_paths[k][0]
    elif len(arrow_paths[k]) == 2:
        new_arrow_path_combinations = []
        for apc in arrow_path_combinations:
            apc1 = apc.copy()
            apc2 = apc.copy()
            apc1[k] = arrow_paths[k][0]
            apc2[k] = arrow_paths[k][1]
            new_arrow_path_combinations.append(apc1)
            new_arrow_path_combinations.append(apc2)
        arrow_path_combinations = new_arrow_path_combinations.copy()
    else:
        print("This should never happen.")

def create_seq_map(seq, arrow_path):
    '''Convert sequence into its hash map of counts'''
    seq = "A" + seq
    seq_dict = {}
    for i, _ in enumerate(seq[:-1]):
        new_s = arrow_path[(seq[i], seq[i+1])]
        if new_s not in seq_dict:
            seq_dict[new_s] = 1
        else:
            seq_dict[new_s] += 1
    return seq_dict


def iterate_seq_map(seq_dict, arrow_path):
    '''Go through each string in a hash map and create new seq map, and then combine.'''
    output_seq_dict = {}
    for seq in seq_dict:
        new_seq_map = create_seq_map(seq, arrow_path)
        for k in new_seq_map:
            if k not in output_seq_dict:
                output_seq_dict[k] = 0
            output_seq_dict[k] += seq_dict[seq] * new_seq_map[k]
    return output_seq_dict

In [6]:
# 21b
t0 = time.time()
sum_complexities = 0
min_keypresses = {} # Format: k=(robot id, from, to), v=minimum key strokes
for sequence in sequences:
    min_length = -1
    for ap in arrow_path_combinations:
        # Calculate num pad sequences
        seqs = get_legal_solutions(sequence, True)

        # Using hash map to come up with combinations
        for seq in seqs:
            seq_dict = create_seq_map(seq, ap)

            for _ in range(25):
                seq_dict = iterate_seq_map(seq_dict, ap)

            total_length = sum(seq_dict.values())

            if total_length < min_length or min_length < 0:
                min_length = total_length
    
    # Calculate complexity
    numeric_part = int(re.findall(r"(\d+)", sequence)[0])
    sum_complexities += numeric_part * min_length
    print(sequence, numeric_part, min_length)
print("21b:", sum_complexities, f" in {time.time() - t0} sec.")

129A 129 90594397580
974A 974 85006969638
805A 805 86475783012
671A 671 90750571882
386A 386 86475783008
21b: 258369757013802  in 0.4017040729522705 sec.
